In [352]:
from sorf_query import *

In [353]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 200

In [ ]:
GENOME_REFERENCE_PATH = 's3://velia-annotation-dev/genomes/hg38/GRCh38.p13.genome.fa.gz'

In [349]:
with open('test_vtx.txt') as fhandle:
    ids = [int(i.replace('VTX-', '')) for i in fhandle.readlines()]
ids[:5]

[7082, 7146, 7396, 7541, 9335]

In [340]:
session = base.Session()

In [350]:
orfs = session.query(Orf).filter(Orf.id.in_(ids)).all()
current_orf = orfs[0]
missing_orfs = set(ids) - set([i.id for i in orfs])
missing_orfs

set()

In [342]:
import pyfaidx
import fsspec
from tqdm import tqdm
import os

if not os.path.exists('reference.fa'):
    with smart_open.open(GENOME_REFERENCE_PATH) as fhandle:
        with open('reference.fa', 'w') as f:
            for line in tqdm(fhandle.readlines()):
                f.write(line)
reference = pyfaidx.Fasta('reference.fa')

In [ ]:
!aws s3 cp s3://velia-annotation-dev/refseq/GCA_000001405.15_GRCh38_full_analysis_set.refseq_annotation.gff .

In [ ]:
!aws s3 ls s3://velia-annotation-dev/refseq/GCA_000001405.15_GRCh38_full_analysis_set.refseq_annotation.gff

                           PRE mRNA_Prot_230110/
2023-04-08 05:49:43 1409662270 GCA_000001405.15_GRCh38_full_analysis_set.refseq_annotation.expanded.gff
2023-04-08 05:49:44 1470041000 GCA_000001405.15_GRCh38_full_analysis_set.refseq_annotation.expanded.ids.gff
2023-04-08 05:49:44 1307343403 GCA_000001405.15_GRCh38_full_analysis_set.refseq_annotation.gff
2023-01-11 05:58:51   57318875 GCA_000001405.15_GRCh38_full_analysis_set.refseq_annotation.gff.gz
2023-01-11 05:58:52      79219 GRCh38_latest_assembly_report.txt
2023-01-11 05:58:51   59486550 GRCh38_latest_clinvar.vcf.gz
2023-01-11 05:58:51  796766480 GRCh38_latest_dbVar.gvf.gz
2023-04-08 05:49:44 1364066134 GRCh38_latest_genomic.gff
2023-01-11 05:58:51   56923273 GRCh38_latest_genomic.gff.gz
2023-01-11 05:58:51  160943050 GRCh38_latest_protein.gpff.gz


In [ ]:
# deduped_conservation = pd.read_parquet('deduped_phylocsf_length_filtered.parq')
transcripts = Fasta('./gencode.v42.transcripts.fa')

def find_seq_substring(query, target_dict):
    return [k for k, s in target_dict.items() if query.lower() in s[:].seq.lower()]

In [26]:
attributes = {
    'chr': current_orf.assembly.ucsc_style_name,
    'vtx': f"VTX-{str(current_orf.id).zfill(7)}",
    'strand': current_orf.strand,
    'start': current_orf.start,
    'end': current_orf.end,
    'nucl': current_orf.nt_seq,
    'aa': current_orf.aa_seq,
}
attributes

{'chr': 'chr17',
 'vtx': 'VTX-0002499',
 'strand': '+',
 'start': 41238052,
 'end': 41238531,
 'nucl': '',
 'aa': 'MTHCCSPCCQPTCCRTTCWKPTTVTTCSSTPCCQPSCCVSSCCQPCCRPTCCQNTCCQPICVTSCCQPSCCSTPCCQPTCCGQTSCGSSCGQSSSCAPVYCRRTCYHPTTVCLPGCLNQSCGSNCCQPCCRPACCETTCCRTTCFQPTCVSSCCQPSCC'}

In [ ]:
assembly_ids = {}

for assembly in session.query(base.Assembly).all():
    if assembly.ucsc_style_name.startswith('chr') and len(assembly.ucsc_style_name) < 6:
        assembly_ids[assembly.ucsc_style_name] = assembly
    else:
        assembly_ids[assembly.genbank_accession] = assembly

In [ ]:
assembly_ids

In [17]:
from Bio import SeqIO

In [21]:
with smart_open.open('s3://velia-annotation-dev/gencode/v42/gencode.v42.transcripts.fa.gz') as f:
    seqs = SeqIO.to_dict(SeqIO.parse(f, 'fasta'))

In [24]:
# seqs.keys()

# Map transcripts

In [253]:
targets = {
    'VTX-0851390': 'ATGGCCGAGAGGCCGGGGCCTCCGGGCGGCGCCGTGTCCGCGACCGCGTACCCTGACACCCCCGCGGAATTCCCTCCGCACCTCCAGGCGGGTGCGATGCGGCGCCGCTTTTGGGGCGTATTCAACTGTCTGTGCGCCGGCGCGTTCGGGGCCCTGGCCGCCGCCTCCGCCAAGCTGGCCTTCGGCAGCGAGGTGAGCATGGGTTTATGCGTCTTAGGCATTATTGTGATGGCGAGCACCAATTCTCTGATGTGGACCTTCTTTAGCCGGGGCCTCAGTTTCTCCATGTCTTCAGCCATTGCATCTGTCACAGTGACTTTTTCAAATATCCTCAGCTCGGCCTTCCTGGGCTATGTGCTGTATGGAGAGTGCCAGGAGGTCTTGTGGTGGGGAGGAGTGTTCCTTATTCTCTGCGGACTCACCCTAATCCACAGGAAGCTCCCACCCACCTGGAAGCCCCTTCCACACAAGCAGCAG',
    'VTX-0081996': 'ATGGCGGATGTGTCAGAGAGGACACTGCAGTTGTCCGTGCTAGTAGCCTTCGCTTCTGGAGTACTCCTGGGCTGGCAGGCGAACCGACTGCGGAGGCGCTACTTGGACTGGAGGAAAAGGAGGCTGCAGGACAAGCTGGCGGCGACGCAGAAGAAGCTGGACCTGGCC',
    'VTX-0087927': 'ATGAACGGCTCTCAGGCGGGCGCCGCGGCTCAGGCCGCCTGGCTGAGCTCCTGCTGTAACCAGTCGGCGTCGCCGCCGGAGCCCCCCGAGGGGCCGCGCGCGGTGCAGGCGGTGGTGCTCGGCGTGCTGTCCCTGCTGGTGCTTTGCGGGGTCCTGTTCCTGGGCGGCGGCCTCCTCCTCCGCGCCCAGGGCCTGACAGCGCTGCTGACCCGCGAGCAGCGCGCGTCCCGCGAGCCCGAGCCGGGCAGTGCCAGCGGAGAGGACGGCGACGACGACTCC',
    'VTX-0085840': 'ATGCTCCTGGGGAGTCTGTGGGGAAGATGCCATCCAGGGCGCTGTGCGCTCTTCCTCATCCTCGCCCTCCTGCTGGACGCGGTCGGCCTGGTCCTTTTGCTGCTGGGGATCTTGGCCCCCCTGAGTTCCTGGGACTTCTTCATCTACACAGGTGCCCTGATCCTGGCTCTCAGCCTACTGCTCTGGATCATCTGGTATTCCCTCAACATTGAGGTGTCTCCTGAAAAACTGGACCTG',
    'VTX-0087707': 'ATGACCTCCTGGCCAGGTGGCAGCTTTGGCCCTGACCCGCTCCTGGCCCTGCTGGTGGTGATCCTGCTAGCACGCCTCATCCTGTGGTCCTGCCTCGGGACCTACATCGACTACAGACTGGCCCAGCGGCGGCCCCAGAAACCCAAGCAGGAC',
}
for i, s in targets.items():
    hit = find_seq_substring(s, transcripts)
    print(i, hit)

VTX-0851390 ['ENST00000302392.5|ENSG00000169964.8|OTTHUMG00000133092.3|OTTHUMT00000256750.3|TMEM42-201|TMEM42|977|protein_coding|']
VTX-0081996 ['ENST00000611969.5|ENSG00000175701.11|OTTHUMG00000131194.5|OTTHUMT00000477505.2|MTLN-203|MTLN|427|protein_coding|', 'ENST00000414416.2|ENSG00000175701.11|OTTHUMG00000131194.5|OTTHUMT00000253918.4|MTLN-201|MTLN|2051|protein_coding|', 'ENST00000426713.1|ENSG00000175701.11|OTTHUMG00000131194.5|OTTHUMT00000338107.3|MTLN-202|MTLN|461|protein_coding|']
VTX-0087927 ['ENST00000546390.2|ENSG00000284791.2|OTTHUMG00000169615.4|OTTHUMT00000405071.2|SMIM41-201|SMIM41|1430|protein_coding|']
VTX-0085840 ['ENST00000581851.2|ENSG00000263429.4|OTTHUMG00000132847.7|OTTHUMT00000256321.4|TMEM238L-202|TMEM238L|1535|protein_coding|']
VTX-0087707 ['ENST00000641568.1|ENSG00000284713.2|OTTHUMG00000172974.2|OTTHUMT00000491249.1|SMIM38-202|SMIM38|2695|protein_coding|', 'ENST00000686937.1|ENSG00000284713.2|OTTHUMG00000172974.2|-|SMIM38-204|SMIM38|5447|protein_coding|', 'E

In [251]:
import gzip
import fsspec
from pyfaidx import Fasta
import pandas as pd
from tqdm import tqdm

# deduped_conservation = pd.read_parquet('deduped_phylocsf_length_filtered.parq')
transcripts = Fasta('./gencode.v42.transcripts.fa')

def find_seq_substring(query, target_dict):
    return [k for k, s in target_dict.items() if query.lower() in s[:].seq.lower()]

def extract_sequence(reference, chrom, strand, block_starts, block_sizes):
    s = reference[chrom]
    sequence = ''
    for start, length in zip(block_starts, block_sizes):
        end = start + (3*(length-1)) + 3
        sequence += str(s[start : end])
    sequence+=str(s[end:end+3])
    return sequence

def pfunc(row):
    ix = row['qName']
    starts = tuple(map(int, row['tStarts'].strip(',').split(',')))
    sizes = tuple(map(int, row['blockSizes'].strip(',').split(',')))
    s = extract_sequence(genes, row['tName'], row['strand'], starts, sizes)
    # print(row['strand'], s[:3], s[-3:])
    # blat_sequences[row['qName']] = s
    matches = find_seq_substring(s, transcripts)
    if len(matches)>0:
        transcript_names = [i.split('|')[0].split('.')[0] for i in matches]
        gene_names = [i.split('|')[5] for i in matches]
        return ix, {'transcripts': transcript_names, 'genes': gene_names, 'sequence': s}
    else:
        return ix, {'transcripts': [], 'genes': [], 'sequence': s}
    
# import multiprocessing as mp
# with mp.Pool(120) as ppool:
#     annotations = {}
#     ixes, rows = list(zip(*deduped_conservation.iterrows()))
#     for r in tqdm(ppool.imap(pfunc, rows), total=50000):#, total=signalp_and_conserved.shape[0]):
#         annotations[r[0]] = r[1]